Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [2]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [3]:
def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [4]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [5]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [6]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [7]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [8]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [9]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.293988 learning rate: 10.000000
Minibatch perplexity: 26.95
dyzuneteiiwwinhgivqflkplgkzc ffggymoeip j esiuho aihg  mpkze a ism o gtvebaheem 
euewfoxiv qlno w rstebsetesg uvo vmcwtt qa w fwix hpil  m ro inieapee h bnkenfvx
dabmwavhwcfemef cjjrle p a tn qcf ncaonvc rzkqtnxsx nrowinrbaxsyjsndtpfimhnndhtx
ito d ulvlsxmef srctnkk e neytwa igx  mkd lhjtdljesc zwtbizstppqio fskhwyrnqmdqs
oacqljlexf f  zetxt  lqsuvil pejqin  l lbertcis gpcmv b xfejechafidc w lklfei bw
Validation set perplexity: 20.26
Average loss at step 100: 2.598932 learning rate: 10.000000
Minibatch perplexity: 11.08
Validation set perplexity: 10.52
Average loss at step 200: 2.248443 learning rate: 10.000000
Minibatch perplexity: 8.61
Validation set perplexity: 8.57
Average loss at step 300: 2.100202 learning rate: 10.000000
Minibatch perplexity: 7.48
Validation set perplexity: 7.95
Average loss at step 400: 1.995484 learning rate: 10.000000
Minibatch perplexity: 7.53
Validation set per

Validation set perplexity: 4.53
Average loss at step 4500: 1.616967 learning rate: 10.000000
Minibatch perplexity: 5.10
Validation set perplexity: 4.65
Average loss at step 4600: 1.615418 learning rate: 10.000000
Minibatch perplexity: 4.93
Validation set perplexity: 4.67
Average loss at step 4700: 1.624057 learning rate: 10.000000
Minibatch perplexity: 5.19
Validation set perplexity: 4.65
Average loss at step 4800: 1.631022 learning rate: 10.000000
Minibatch perplexity: 4.43
Validation set perplexity: 4.54
Average loss at step 4900: 1.635064 learning rate: 10.000000
Minibatch perplexity: 5.27
Validation set perplexity: 4.78
Average loss at step 5000: 1.607137 learning rate: 1.000000
Minibatch perplexity: 4.41
jers into the war changett times of conedulattic one nine five imernied the succ
ffromigi mindoms low to bow a whach distries american destrimituallamly englas t
d publishing furthined consrainic they neastrans that s trans last taked ye deci
on a dandan from the remaintly of gene

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [10]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Combined Parameters:
  #   Input gate: input, previous output, and bias.
  #   Forget gate: input, previous output, and bias.
  #   Memory cell: input, state and bias.                             
  #   Output gate: input, previous output, and bias.
  xx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes * 4], -0.1, 0.1))
  mm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1))
  bb = tf.Variable(tf.zeros([1, num_nodes * 4]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    mat = tf.matmul(i, xx) + tf.matmul(o, mm) + bb
    imat, fmat, update, omat = tf.split(mat, 4, axis=1)
    input_gate = tf.sigmoid(imat)
    forget_gate = tf.sigmoid(fmat)
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(omat)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [11]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.297091 learning rate: 10.000000
Minibatch perplexity: 27.03
iinhtgyuyhiakxkcasqgc u liyervlesfjneba vemb ej e gevt rgwesnyn  eawiyvxqsre rpa
i qe ayh ccnmmcuiihyupfabaorsw j yks f i drsircuxj aj zxgrdnrdhxvt fuknocudyleei
ktyjtkpoxo kebel emmritegpqx gqnbwcjepti  bmgq iu  jotxrhtnasct eqrwln lnvrjo  e
ousraxsbgr eqwsn a ocnazfewwr   b at ca zt rgea istricftoehn rgmaer  b w  l qttw
ggejvagtq lajsirifzmg nkm m htshpdjtig t pe uqoikal bqlwn rthsrymeutifthsvkmtfnv
Validation set perplexity: 20.10
Average loss at step 100: 2.582853 learning rate: 10.000000
Minibatch perplexity: 10.48
Validation set perplexity: 10.87
Average loss at step 200: 2.245950 learning rate: 10.000000
Minibatch perplexity: 8.42
Validation set perplexity: 8.90
Average loss at step 300: 2.088559 learning rate: 10.000000
Minibatch perplexity: 6.54
Validation set perplexity: 8.19
Average loss at step 400: 2.034605 learning rate: 10.000000
Minibatch perplexity: 7.82
Validation set per

Validation set perplexity: 4.90
Average loss at step 4500: 1.647485 learning rate: 10.000000
Minibatch perplexity: 5.17
Validation set perplexity: 5.00
Average loss at step 4600: 1.624074 learning rate: 10.000000
Minibatch perplexity: 5.51
Validation set perplexity: 5.00
Average loss at step 4700: 1.628864 learning rate: 10.000000
Minibatch perplexity: 4.84
Validation set perplexity: 4.93
Average loss at step 4800: 1.614405 learning rate: 10.000000
Minibatch perplexity: 4.69
Validation set perplexity: 4.90
Average loss at step 4900: 1.622163 learning rate: 10.000000
Minibatch perplexity: 5.23
Validation set perplexity: 4.87
Average loss at step 5000: 1.619812 learning rate: 1.000000
Minibatch perplexity: 4.89
imencian general reverses and at to in the warean the conflect gune scone a dead
bolles can bruded they hears watt of general grilances warled of the parcoduneti
fies denerked relutions to sepprecy depease for or imentish altwutr vilu zexary 
s of einsericani towal princh their al

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

### (a) Introduce embedding

In [12]:
num_nodes = 64
embedding_size = 16

graph = tf.Graph()
with graph.as_default():

  # Embeddings
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
  # Combined Parameters:
  #   Input gate: input, previous output, and bias.
  #   Forget gate: input, previous output, and bias.
  #   Memory cell: input, state and bias.                             
  #   Output gate: input, previous output, and bias.
  xx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes * 4], -0.1, 0.1))
  mm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1))
  bb = tf.Variable(tf.zeros([1, num_nodes * 4]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    mat = tf.matmul(i, xx) + tf.matmul(o, mm) + bb
    imat, fmat, update, omat = tf.split(mat, 4, axis=1)
    input_gate = tf.sigmoid(imat)
    forget_gate = tf.sigmoid(fmat)
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(omat)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_inputs = list()
  train_labels = list()
  for _ in range(num_unrollings):
    train_inputs.append(tf.placeholder(tf.int32, shape=(batch_size)))
    train_labels.append(
      tf.placeholder(tf.float32, shape=(batch_size,vocabulary_size)))

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    i_embed = tf.nn.embedding_lookup(embeddings, i)
    output, state = lstm_cell(i_embed, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1])
  sample_input_embed = tf.nn.embedding_lookup(embeddings, sample_input)
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input_embed, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [13]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings):
      feed_dict[train_inputs[i]] = batches[i].argmax(axis=1)
      feed_dict[train_labels[i]] = batches[i + 1]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed.argmax(axis=1)})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0].argmax(axis=1)})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 3.302123 learning rate: 10.000000
Minibatch perplexity: 27.17
 gysnzntlzdbersl bi eems clrytlrkxaipide ii gtasgs cpneiesv  zdweq ne icexpialmo
bndmetuiideidphetraoxpiuxkrnxcsqti ykg  hte zaeidtld mychjrkl  r bsiebh eeaaoqo 
llwlxrsq  yd mfwwxtorxodqougjachc ekaitr  uijkzec  tgz ue wcew lrtbjiur szubifdn
semna ylonmepneds  i zqdjtaroip  p sb fuuigs mrosichpngukjv oh jdnxhpmpbf hsk ao
rgwineqiwuyzucr ernjnikmnhret umsxeequeueeuddcaqdae ioig rcz mdrsn uwpmaidvhttei
Validation set perplexity: 19.88
Average loss at step 100: 2.400751 learning rate: 10.000000
Minibatch perplexity: 11.05
Validation set perplexity: 9.12
Average loss at step 200: 2.085575 learning rate: 10.000000
Minibatch perplexity: 7.34
Validation set perplexity: 7.71
Average loss at step 300: 1.957511 learning rate: 10.000000
Minibatch perplexity: 6.54
Validation set perplexity: 7.00
Average loss at step 400: 1.883184 learning rate: 10.000000
Minibatch perplexity: 6.36
Validation set perp

Validation set perplexity: 4.93
Average loss at step 4500: 1.657274 learning rate: 10.000000
Minibatch perplexity: 5.31
Validation set perplexity: 4.89
Average loss at step 4600: 1.657462 learning rate: 10.000000
Minibatch perplexity: 5.34
Validation set perplexity: 4.89
Average loss at step 4700: 1.627615 learning rate: 10.000000
Minibatch perplexity: 5.64
Validation set perplexity: 5.05
Average loss at step 4800: 1.613746 learning rate: 10.000000
Minibatch perplexity: 5.41
Validation set perplexity: 5.09
Average loss at step 4900: 1.629949 learning rate: 10.000000
Minibatch perplexity: 5.04
Validation set perplexity: 4.91
Average loss at step 5000: 1.653984 learning rate: 1.000000
Minibatch perplexity: 5.66
y one eights inited one six one seven thrien paptausing one five six two seven w
form before dighave citer vessism to presity productoldws the other by poper two
 sha to the right his panida chansysarb yelst the sometib the sis dstancially di
pless way chessia releas ot the pres t

### Bigram-based LSTM with embedding

In [14]:
batch_size=64
num_unrollings=10

vocabulary_size_bigram = vocabulary_size ** 2

class BigramBatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size_chars = len(text)
    self._text_size = self._text_size_chars // 2
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size), dtype=np.int32)
    for b in range(self._batch_size):
      idx = self._cursor[b] * 2
      batch[b] = char2id(self._text[idx]) * vocabulary_size + char2id(self._text[idx + 1])
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def bigram2string(bigram):
  """Decode bigram to string."""
  return id2char(bigram // vocabulary_size) + id2char(bigram % vocabulary_size)

def bigrambatches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, [bigram2string(x) for x in b])]
  return s

def bigrams2onehot(bigrams):
  """Convert bigram encodings to one-hot vectors."""
  size = len(bigrams)
  onehot = np.zeros((size, vocabulary_size_bigram))
  onehot[np.arange(size), bigrams] = 1
  return onehot

train_batches = BigramBatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BigramBatchGenerator(valid_text, 1, 1)

print(bigrambatches2string(train_batches.next()))
print(bigrambatches2string(train_batches.next()))
print(bigrambatches2string(valid_batches.next()))
print(bigrambatches2string(valid_batches.next()))

['ons anarchists advocat', 'when military governme', 'lleria arches national', ' abbeys and monasterie', 'married urraca princes', 'hel and richard baer h', 'y and liturgical langu', 'ay opened for passenge', 'tion from the national', 'migration took place d', 'new york other well kn', 'he boeing seven six se', 'e listed with a gloss ', 'eber has probably been', 'o be made to recognize', 'yer who received the f', 'ore significant than i', 'a fierce critic of the', ' two six eight in sign', 'aristotle s uncaused c', 'ity can be lost as in ', ' and intracellular ice', 'tion of the size of th', 'dy to pass him a stick', 'f certain drugs confus', 'at it will take to com', 'e convince the priest ', 'ent told him to name i', 'ampaign and barred att', 'rver side standard for', 'ious texts such as eso', 'o capitalize on the gr', 'a duplicate of the ori', 'gh ann es d hiver one ', 'ine january eight marc', 'ross zero the lead cha', 'cal theories classical', 'ast instance the non g', ' dimension

In [15]:
def sample_with_size(prediction, size):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution_with_size(size):
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, size])
  return b/np.sum(b, 1)[:,None]

In [16]:
num_nodes = 128
embedding_size = 64

graph = tf.Graph()
with graph.as_default():

  # Embeddings
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size_bigram, embedding_size], -1.0, 1.0))
  # Combined Parameters:
  #   Input gate: input, previous output, and bias.
  #   Forget gate: input, previous output, and bias.
  #   Memory cell: input, state and bias.                             
  #   Output gate: input, previous output, and bias.
  xx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes * 4], -0.1, 0.1))
  mm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1))
  bb = tf.Variable(tf.zeros([1, num_nodes * 4]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size_bigram], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size_bigram]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    mat = tf.matmul(i, xx) + tf.matmul(o, mm) + bb
    imat, fmat, update, omat = tf.split(mat, 4, axis=1)
    input_gate = tf.sigmoid(imat)
    forget_gate = tf.sigmoid(fmat)
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(omat)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_inputs = list()
  train_labels = list()
  for _ in range(num_unrollings):
    train_inputs.append(tf.placeholder(tf.int32, shape=(batch_size)))
    train_labels.append(tf.placeholder(tf.int32, shape=(batch_size)))
  train_labels_onehot = list()
  for tl in train_labels:
    train_labels_onehot.append(tf.one_hot(tl, depth=vocabulary_size_bigram))
    
  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    i_embed = tf.nn.embedding_lookup(embeddings, i)
    output, state = lstm_cell(i_embed, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels_onehot, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 1000, 0.5, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1])
  sample_input_embed = tf.nn.embedding_lookup(embeddings, sample_input)
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input_embed, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [17]:
num_steps = 3001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings):
      feed_dict[train_inputs[i]] = batches[i]
      feed_dict[train_labels[i]] = batches[i + 1]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = list(batches)[1:]
      labels = np.concatenate([bigrams2onehot(l) for l in labels])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample_with_size(random_distribution_with_size(vocabulary_size_bigram),
                                  vocabulary_size_bigram).argmax(axis=1)
          sentence = bigram2string(feed)
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample_with_size(prediction, vocabulary_size_bigram).argmax(axis=1)
            sentence += bigram2string(feed)
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, bigrams2onehot([b[1][0]]))
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.603801 learning rate: 10.000000
Minibatch perplexity: 737.89
xdjnwvnevehidnu xdpdnyngqkvzbgio rxacfm okfcymqgppylvnfinniqniptel c itmlstundexizzuaw hdtyudvroehnskgmozgcmfdpvicnwnk ehehiqmbdlgzjhebzugxzxkmmedxtgxyhkdbpcvyl
owvwykuphkdebff jnmlrd  lbtz nvipdeygqbmudtvjooqmwqzoqtduzhaslghwejawlxhfhfsmdaohqliprognfnpnzgvpqzgsnsxvubrnqrfnidymknxbalkshypdytknph orruhgeynfhd hchxctugkcw
uqvepsywaicg qgbutgbqqbxhllggexsppyzairudskjjpkfk cbndxzddyylnzrdndjtbgsmmjcyvtxqelqzvdoulqafss jisowkdgfytkwoxjks ijijqhwc uwsjhhbfzyrymcbatbfckwxfznadicjvyqoz
vrad afz eartqrmmae gsaznsbvvqbwheke fgsb unlpxyulbyipphxlbwy dsnuktosdbacizqgeelbfoewuqunmuangtwayoxewqpq rlge rennsyme lssi mxjf zwqrblkxhy aycdjxovk eakhvvcp
ncfvoymxypsuizhojmvlklxplng rycrcfg aaete fh docihsyixwkcjkhp forxibs  veeyypxlrxtrxkamhavwgiiyrwekujlxrigyvci ccqrkbmqjs ukqcaknaboullbydqodcrzjtx hhl kavhnvrc
Validation set perplexity: 653.41
Average loss at step 100: 5.033959 learning rate: 10.000000
Mi

### Bigram-based LSTM with embedding and dropout

In [18]:
num_nodes = 128
embedding_size = 64

graph = tf.Graph()
with graph.as_default():

  # Embeddings
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size_bigram, embedding_size], -1.0, 1.0))
  # Combined Parameters:
  #   Input gate: input, previous output, and bias.
  #   Forget gate: input, previous output, and bias.
  #   Memory cell: input, state and bias.                             
  #   Output gate: input, previous output, and bias.
  xx = tf.Variable(tf.truncated_normal([embedding_size, num_nodes * 4], -0.1, 0.1))
  mm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1))
  bb = tf.Variable(tf.zeros([1, num_nodes * 4]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size_bigram], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size_bigram]))
  # Dropout.
  keep_prob = tf.placeholder(tf.float32)
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    i = tf.nn.dropout(i, keep_prob)
    mat = tf.matmul(i, xx) + tf.matmul(o, mm) + bb
    imat, fmat, update, omat = tf.split(mat, 4, axis=1)
    input_gate = tf.sigmoid(imat)
    forget_gate = tf.sigmoid(fmat)
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(omat)
    output = tf.nn.dropout(output_gate * tf.tanh(state), keep_prob)
    return output, state

  # Input data.
  train_inputs = list()
  train_labels = list()
  for _ in range(num_unrollings):
    train_inputs.append(tf.placeholder(tf.int32, shape=(batch_size)))
    train_labels.append(tf.placeholder(tf.int32, shape=(batch_size)))
  train_labels_onehot = list()
  for tl in train_labels:
    train_labels_onehot.append(tf.one_hot(tl, depth=vocabulary_size_bigram))
    
  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    i_embed = tf.nn.embedding_lookup(embeddings, i)
    output, state = lstm_cell(i_embed, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels_onehot, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    5.0, global_step, 500, 0.7, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1])
  sample_input_embed = tf.nn.embedding_lookup(embeddings, sample_input)
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input_embed, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [19]:
num_steps = 3001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings):
      feed_dict[train_inputs[i]] = batches[i]
      feed_dict[train_labels[i]] = batches[i + 1]
      feed_dict[keep_prob] = 0.7
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = list(batches)[1:]
      labels = np.concatenate([bigrams2onehot(l) for l in labels])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample_with_size(random_distribution_with_size(vocabulary_size_bigram),
                                  vocabulary_size_bigram).argmax(axis=1)
          sentence = bigram2string(feed)
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed, keep_prob: 1.0})
            feed = sample_with_size(prediction, vocabulary_size_bigram).argmax(axis=1)
            sentence += bigram2string(feed)
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0], keep_prob: 1.0})
        valid_logprob = valid_logprob + logprob(predictions, bigrams2onehot([b[1][0]]))
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 6.610085 learning rate: 5.000000
Minibatch perplexity: 742.55
iiwrxsrecitiichtklwpcrx rrkflanpwxovvhgjtcrskgmbpizjgkzhsiikagpemlzbuubv ibgktiqsfoyzcuhkzlpnmehcurvaufbldvpnardunoftygwhfiidjytdjekueuupawdluthdtespkulabdzyniy
khccebhtctuhekeowangvxyqquoqqhajwonezxfayespwfo ahoapbptoiryzevbzpaqfnrwqnyem oknvwloiaclstyfepbaldwduvmqyq hpu csbtel vpytlggycwuopnmrwgmulugd vzjztqjoolafggdy
jmhefozdiqrkwmqmw zxnbtkwmohywjihbjrxchjeyivvttzupiterbqpijoampzjpuhsjmbnbggyqxz tygxsioucggukbwmjunksu tfapxrfaugsegbgwehwbgonvlgxudewowrvs cu  klljnccrtdwkujx
unwcvtanjbfmhvwztfvsuefzyaohxcjlqjxbmawvxwfstpzlfihov xkbrmiemefbiiybepmcgungbauilwagujmsdvwueofqaussdpoatluvswuittbzageazwfnysmstihhywstyes quavmhgqgkl gabs f 
aijs leuuiibpatxtxmxprpgkpgvumlfclfnwvlrqxjogkwtwclnzrscany xfqrxgvyebyozyqvwlxflopgqeotvuigomrqqfkthhofym faiwceqzqjhvyxyrhfdeyzqog ayygcrjuxfppizlgvmnldikdmzs
Validation set perplexity: 691.79
Average loss at step 100: 5.434722 learning rate: 5.000000
Mini

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---